In [1]:
import enum
import math
from random import randint
from itertools import groupby
import numpy as np
import random as rnd
!pip install mido
from mido import MidiFile, Message, MidiTrack
import music21
import copy
from operator import add
!pip install pretty_midi
import pretty_midi
from collections import Counter

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.1/51.1 KB 1.7 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 59.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pretty_midi: filename=pretty_midi-0.2.10-py3-none-any.whl size=5592303 sha256=b3f067081f10ce8cda40f32ad5954a946c77d55860d3fb31b0656e5a7bd24ad6
  Stored in directory: /root/.cache/pip/wheels/75/ec/20/b8e937a5bcf1de547ea5ce465db7de7f6761e15e6f0a01e25f
Successfully built pretty_midi


In [2]:
filepath = '/content/audio.mid'

In [3]:
def parseMidi(filepath):
  notes_list = []
  midi_data = pretty_midi.PrettyMIDI(filepath)
  for instrument in midi_data.instruments:
    for note in instrument.notes:
      notes_list.append([note.pitch, note.start, note.end])
  return notes_list
print(parseMidi(filepath))

[[58, 0.03488372093023256, 0.06976744186046512], [70, 0.023255813953488372, 0.06976744186046512], [93, 0.023255813953488372, 0.08139534883720931], [71, 0.9534883720930233, 0.9883720930232558], [74, 0.0, 1.7325581395348837], [67, 0.9534883720930233, 2.011627906976744], [58, 2.0232558139534884, 2.058139534883721], [51, 2.0348837209302326, 2.0697674418604652], [69, 2.011627906976744, 3.0348837209302326], [64, 3.0348837209302326, 3.058139534883721], [74, 3.0232558139534884, 3.058139534883721], [89, 3.058139534883721, 3.13953488372093], [50, 4.034883720930233, 4.069767441860465], [58, 4.034883720930233, 4.069767441860465], [70, 3.0232558139534884, 4.988372093023256], [74, 4.0, 4.988372093023256], [71, 4.988372093023256, 5.011627906976744], [70, 5.023255813953488, 5.058139534883721], [74, 5.011627906976744, 5.406976744186046], [67, 4.988372093023256, 5.965116279069767], [73, 5.976744186046512, 6.0], [69, 5.965116279069767, 6.976744186046512], [70, 6.976744186046512, 9.05813953488372], [70, 9

In [4]:
def midiStringtoInt(note):
   notes = [["A"], ["A#", "Bb"], ["B"], ["C"], ["C#", "Db"], ["D"], ["D#", "Eb"], ["E"], ["F"], ["F#", "Gb"], ["G"],["G#", "Ab"]]
   for i in range(len(notes)):
     if note in notes[i]:
       return i
   return None


In [5]:
score = music21.converter.parse(filepath)
notes_list = parseMidi(filepath)
key = score.analyze('key')
print(key.tonic, key.mode)

G minor


In [6]:
mid = MidiFile(filepath, clip=True)
for track in mid.tracks:
  print(track)

MidiTrack([
  MetaMessage('set_tempo', tempo=500000, time=0),
  Message('program_change', channel=0, program=0, time=0),
  Message('note_on', channel=0, note=74, velocity=90, time=0),
  Message('note_on', channel=0, note=70, velocity=23, time=2),
  Message('note_on', channel=0, note=93, velocity=24, time=0),
  Message('note_on', channel=0, note=58, velocity=24, time=1),
  Message('note_off', channel=0, note=58, velocity=64, time=3),
  Message('note_off', channel=0, note=70, velocity=64, time=0),
  Message('note_off', channel=0, note=93, velocity=64, time=1),
  Message('note_on', channel=0, note=67, velocity=80, time=75),
  Message('note_on', channel=0, note=71, velocity=28, time=0),
  Message('note_off', channel=0, note=71, velocity=64, time=3),
  Message('note_off', channel=0, note=74, velocity=64, time=64),
  Message('note_off', channel=0, note=67, velocity=64, time=24),
  Message('note_on', channel=0, note=69, velocity=84, time=0),
  Message('note_on', channel=0, note=58, velocity=2

In [11]:
CHORD_Period = 2 * mid.ticks_per_beat
AMOUNT_OF_CHORDS = math.ceil(mid.length)
print(AMOUNT_OF_CHORDS)


17


In [7]:
chord_circle = [['C', 'A', 'B'], ['G', 'E', 'F#'], ['D', 'B', 'C#'], ['A', 'F#', 'G#'],                 ['E', 'C#', 'D#'], ['B', 'G#', 'A#'], ['F#', 'D#', 'E#'], ['C#', 'A#', 'C'],                 ['G#', 'F', 'G'], ['D#', 'C', 'D'], ['A#', 'G', 'A'], ['F', 'D', 'E']]

In [8]:
baseOctaveNote = key.tonic.midi - midiStringtoInt(key.tonic.name) - 12

In [9]:
class Chord:
  def __init__(self, note, type):
     major = [0, 4, 7]
     minor = [0, 3, 7]
     sus2 = [0, 2, 7]
     sus4 = [0, 5, 7]
     dim = [0, 3, 6]
     self.chord = [note, note, note]
     if type == 0:
            self.chord = list( map(add, self.chord, major))
     elif type == 1:
            self.chord = list(map(add, self.chord, minor))
     elif type == 2:
            self.chord = list(map(add, self.chord, sus2))
     elif type == 3:
            self.chord = list(map(add, self.chord, sus4))
     elif type == 4:
            self.chord = list(map(add, self.chord, dim))

  def __getitem__(self, key):
    return self.chord[key]

In [12]:
class Chromosome:
  def __init__(self, baseOctaveNote, genes_amount=AMOUNT_OF_CHORDS, genes=None):
    self.length = genes_amount if genes is None else len(genes)
    mode = 0 if key.mode == 'major' else 1
    ind = 0
    for i in range(len(chord_circle)):
      if chord_circle[i][mode] == key.tonic.name:
        ind = i
        break
    self.sus_chords = [Chord(baseOctaveNote + midiStringtoInt(chord_circle[ind][mode]), type=2),
                           Chord(baseOctaveNote + midiStringtoInt(chord_circle[ind][abs(1 - mode)]),
                                 type=2),
                           Chord(baseOctaveNote + midiStringtoInt(chord_circle[(ind + 1) % 12][abs(1 - mode)]),
                                 type=2),
                           Chord(baseOctaveNote + midiStringtoInt(chord_circle[(ind - 1) % 12][abs(1 - mode)]),
                                 type=2),
                           Chord(baseOctaveNote + midiStringtoInt(chord_circle[(ind + 1) % 12][mode]), type=2),
                           Chord(baseOctaveNote + midiStringtoInt(chord_circle[(ind - 1) % 12][mode]), type=2),

                           Chord(baseOctaveNote + midiStringtoInt(chord_circle[ind][mode]), type=3),
                           Chord(baseOctaveNote + midiStringtoInt(chord_circle[ind][abs(1 - mode)]),
                                 type=3),
                           Chord(baseOctaveNote + midiStringtoInt(chord_circle[(ind + 1) % 12][abs(1 - mode)]),
                                 type=3),
                           Chord(baseOctaveNote + midiStringtoInt(chord_circle[(ind - 1) % 12][abs(1 - mode)]),
                                 type=3),
                           Chord(baseOctaveNote + midiStringtoInt(chord_circle[(ind + 1) % 12][mode]), type=3),
                           Chord(baseOctaveNote + midiStringtoInt(chord_circle[(ind - 1) % 12][mode]), type=3)
                           ]

    self.major_minor_dim_chords = [Chord(baseOctaveNote + midiStringtoInt(chord_circle[ind][mode]), type=mode),
                                Chord(baseOctaveNote + midiStringtoInt(chord_circle[ind][abs(1 - mode)]),
                                      type=abs(1 - mode)),
                                Chord(baseOctaveNote + midiStringtoInt(chord_circle[(ind + 1) % 12][abs(1 - mode)]),
                                      type=abs(1 - mode)),
                                Chord(baseOctaveNote + midiStringtoInt(chord_circle[(ind - 1) % 12][abs(1 - mode)]),
                                      type=abs(1 - mode)),
                                Chord(baseOctaveNote + midiStringtoInt(chord_circle[(ind + 1) % 12][mode]), type=mode),
                                Chord(baseOctaveNote + midiStringtoInt(chord_circle[(ind - 1) % 12][mode]), type=mode),

                                Chord(baseOctaveNote + midiStringtoInt(chord_circle[ind][2]), type=4),
                                ]
    if genes is None:
      population = []
      for i in range(self.length):
        population.append(self.chooseRandomChord())
      self.genes = population
    else:
      self.genes = genes
  def chooseRandomChord(self):
    if rnd.random() <= 0.05:
      rand = rnd.randint(0, len(self.sus_chords) - 1)
      return self.sus_chords[rand]
    rand = rnd.randint(0, len(self.major_minor_dim_chords) - 1)
    return self.major_minor_dim_chords[rand]
  def get_genes(self):
    return list(self.genes).copy()

  def __getitem__(self, key):
    return self.genes[key]

  def __len__(self):
    return len(self.genes)

  def __copy__(self):
    cls = self.__class__
    result = cls.__new__(cls)
    result.__dict__.update(self.__dict__)
    return result

  def __setitem__(self, key, value):
    self.__dict__[key] = value


In [13]:
class GeneticAlgo:
  def __init__(self, init_pop_size=500, pop_size=20):
    self.pop_size = pop_size
    self.population = []
    for i in range(init_pop_size):
      self.population.append(Chromosome(baseOctaveNote))
  
  def mutation(self, sol):
    n = len(sol)
    pos = rnd.randint(0, n - 1)
    pos_2 = rnd.randint(0, n - 1)
    result = copy.copy(sol)
    result[pos] = sol.chooseRandomChord()
    result[pos_2] = sol.chooseRandomChord()
    return result

  def swap(self, sol, posA, posB):
    result = copy.copy(sol)
    t = sol[posA]
    result[posA] = sol[posB]
    result[posB] = t
    return result

  
  def crossover(self, solA, solB):
    n = len(solA)
    child = list(copy.copy(solA))
    num_els = np.ceil(n * (rnd.randint(10, 90) / 100))
    start_pnt = rnd.randint(0, n - 2)
    end_pnt = n if int(start_pnt + num_els) > n else int(start_pnt + num_els)
    child[start_pnt:end_pnt] = solB[start_pnt:end_pnt]
    return Chromosome(baseOctaveNote, genes=child)
  

  def fitness_function(self, solution):
        fitness = 0
        time = 0
        prev_note_chord_pair = [None, None]
        for i in range(len(solution)):
            chord = solution[i]
            ind = 4 * i if 4 * i < len(notes_list) else len(notes_list) - 1
            note = notes_list[ind][0]
            if i == 0:
                prev_note_chord_pair = [note, chord[0]]
            else:
                if note > prev_note_chord_pair[0] and chord[0] > prev_note_chord_pair[1] or \
                        note < prev_note_chord_pair[0] and chord[0] < prev_note_chord_pair[1]:
                    fitness += 10
            if abs(chord[0] - prev_note_chord_pair[1]) > 5:
                fitness -= 8
            diffs = [(note - e) % 12 for e in chord]
            if 0 in diffs:
                fitness += 10
            elif (diffs[0] >= 3 and diffs[0] <= 8) or diffs[0] == 11:
                fitness += 10
            else:
                fitness -= 10
            threshold = 6
            border = 0 if i - threshold < 0 else i - threshold
            count = Counter(solution[border:i])
            if count[chord] > 2:
                fitness-=10
            time += 1
            prev_note_chord_pair = [note, chord[0]]
        if solution[-1][0] == key.tonic.midi:
            fitness+=50
        for _, group in groupby(solution):
            count_dups = sum(1 for _ in group)
            if count_dups > 2:
                fitness = -100
        return fitness
  def pickSolution(self, pop_bag):
        fit_bag_evals = self.evaluatePopulationFitness(pop_bag)
        picked = fit_bag_evals["solution"][0]
        while True:
            rnIndex = rnd.randint(0, len(pop_bag) - 1)
            rnPick = fit_bag_evals["fit_wgh"][rnIndex]
            r = rnd.random()
            if r <= rnPick:
                picked = fit_bag_evals["solution"][rnIndex]
                break
        return picked
    
  def evaluatePopulationFitness(self, pop_bag):
        result = {}
        fit_vals = []
        solutions = []
        for solution in pop_bag:
            fit_vals.append(self.fitness_function(solution))
            solutions.append(solution)
        result["fit_vals"] = fit_vals
        fit_prob = [i - min(list(result["fit_vals"])) + 1 for i in list(result["fit_vals"])]
        result["fit_wgh"] = [i / sum(fit_prob) for i in fit_prob]
        result["solution"] = solutions
        return result
  
  def run(self):
        # The initial population
        pop_bag = ga.population
        best_fit_global = None
        best_solution_global = None
        # Iterating over all generations
        for g in range(2000):
            print(f'Generation {g}...')
            # Evaluate fitness for all individuals
            pop_bag_fit = ga.evaluatePopulationFitness(pop_bag)
            # Best individual so far
            best_fit = np.max(pop_bag_fit["fit_vals"])
            best_fit_index = pop_bag_fit["fit_vals"].index(best_fit)
            best_solution = pop_bag_fit["solution"][best_fit_index]
            # at first iteration there's nothing to compare to the solutions
            if g == 0:
                best_fit_global = best_fit
                best_solution_global = best_solution
            # if found new best, save it
            else:
                if best_fit >= best_fit_global:
                    best_fit_global = best_fit
                    best_solution_global = best_solution
            # Create the new population bag
            new_pop_bag = []
            for i in range(self.pop_size):
                # Pick 2 parents from the generation
                pA = ga.pickSolution(pop_bag)
                pB = ga.pickSolution(pop_bag)
                new_element = pA
                # Crossover the parents with probability
                if rnd.random() <= 0.9:
                    new_element = ga.crossover(pA, pB)
                # Mutate the child with probability
                if rnd.random() <= 0.4:
                    new_element = ga.mutation(new_element)
                # Append the child to the bag
                new_pop_bag.append(new_element)
            pop_bag = new_pop_bag
        return  best_solution_global



In [14]:
def saveSolution(solution: Chromosome, path=filepath[:-4]+'-withAccompaniment.mid'):
    VELOCITY = 50
    track = MidiTrack()
    # length of a chord in ticks
    CHORD_TICKS = 2 * mid.ticks_per_beat
    for chord in solution:
        track.append(Message('note_on', note=chord[0], velocity=VELOCITY, time=0))
        track.append(Message('note_on', note=chord[1], velocity=VELOCITY, time=0))
        track.append(Message('note_on', note=chord[2], velocity=VELOCITY, time=0))

        track.append(Message('note_off', note=chord[0],
                             velocity=0, time=CHORD_TICKS))
        track.append(Message('note_off', note=chord[1],
                             velocity=0, time=0))
        track.append(Message('note_off', note=chord[2],
                             velocity=0, time=0))
    mid.tracks.append(track)
    mid.save(path)

# i have different size of population on initialization to increase exploration of chords
ga = GeneticAlgo(init_pop_size=500, pop_size=20)
best_solution = ga.run()
saveSolution(best_solution)
print("DONE")

Generation 0...
Generation 1...
Generation 2...
Generation 3...
Generation 4...
Generation 5...
Generation 6...
Generation 7...
Generation 8...
Generation 9...
Generation 10...
Generation 11...
Generation 12...
Generation 13...
Generation 14...
Generation 15...
Generation 16...
Generation 17...
Generation 18...
Generation 19...
Generation 20...
Generation 21...
Generation 22...
Generation 23...
Generation 24...
Generation 25...
Generation 26...
Generation 27...
Generation 28...
Generation 29...
Generation 30...
Generation 31...
Generation 32...
Generation 33...
Generation 34...
Generation 35...
Generation 36...
Generation 37...
Generation 38...
Generation 39...
Generation 40...
Generation 41...
Generation 42...
Generation 43...
Generation 44...
Generation 45...
Generation 46...
Generation 47...
Generation 48...
Generation 49...
Generation 50...
Generation 51...
Generation 52...
Generation 53...
Generation 54...
Generation 55...
Generation 56...
Generation 57...
Generation 58...
Generat

ValueError: ignored

In [ ]:
filepath2 = '/content/audio-withAccompaniment.mid'
midi_data1 = pretty_midi.PrettyMIDI(filepath2)
notes_list2 = []
for instrument in midi_data1.instruments:
    for note in instrument.notes:
      num = note.pitch
      note = music21.pitch.Pitch(num)
      notes_list2.append(note.name)
print(notes_list2)



['G#', 'E', 'G#', 'C#', 'A', 'F#', 'E-', 'F#', 'B', 'G#', 'F#', 'E', 'E', 'C#', 'F#', 'C#', 'F#', 'E', 'G#', 'F#', 'A', 'C#', 'E', 'C#', 'E', 'G#', 'F#', 'A', 'C#', 'A', 'C#', 'E', 'C#', 'E', 'G#', 'F#', 'A', 'C#', 'E-', 'F#', 'A', 'B', 'E-', 'F#']


In [ ]:
final1 = music21.stream.Score()
final
i1 = music21.stream.Part()
i2 = music21.stream.Part()
i3 = music21.stream.Part()

i1.insert(music21.instrument.Guitar())
i2.insert(music21.instrument.Violin())
i3.insert(music21.instrument.Harp())
solution = Chromosome

for note1 in notes_list2:
  newnote = music21.note.Note(note1)
  newnote.duration.type = 'quarter'
  i1.append(newnote)

  newnote = music21.note.Note(note1)
  newnote.duration.type = 'quarter'
  i2.insert(newnote)

  newnote = music21.note.Note(note1)
  newnote.duration.type = 'quarter'
  i3.append(newnote)

final.insert(0, i1)
#final.insert(0, i2)

final.write("midi", "final.mid")

TypeError: ignored

In [ ]:
!pip install py-midi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.6/90.6 KB 4.4 MB/s eta 0:00:00


In [ ]:
filepath3 = '/content/final.mid'
mid1 = MidiFile(filepath)
mid2 = MidiFile(filepath3)

merged_mid = MidiFile()
merged_mid.ticks_per_beat=mid1.ticks_per_beat

merged_mid.tracks = mid1.tracks + mid2.tracks
merged_mid.save('merged.mid')